In [1]:
import numpy as np
import os
import tensorflow as tf
import pandas as pd
from tensorflow.python.framework import ops

/home/fraz/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/fraz/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/fraz/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/fraz/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:461: FutureWa

In [3]:
# Target location of output
folder_locator = './outputs/'

# Locations of TensorBoard and model save outputs
board_string = folder_locator + 'tensorboard/5_classic/'
checkpoint_string = folder_locator + 'models/5_classic/'
# Number of batches to use in the optimization
batch_size = 25

In [4]:
train = np.loadtxt('unsw_train.csv' , delimiter=',')
test = np.loadtxt('unsw_test.csv' , delimiter=',')

In [5]:
print("train shape " , train.shape)
print("test shape " ,test.shape)

train shape  (82300, 43)
test shape  (60000, 43)


In [6]:
X_train = train[: , :42]
Y_train = train[: , 42:]
X_test =  test[: , :42]
Y_test =  test[: , 42:]

In [7]:
print("x train shape: ",X_train.shape)
print("y train shape: ",Y_train.shape)
print("x test shape: ",X_test.shape)
print("y test shape: ",Y_test.shape)

x train shape:  (82300, 42)
y train shape:  (82300, 1)
x test shape:  (60000, 42)
y test shape:  (60000, 1)


In [8]:
def create_placeholders(n_x, n_y):
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_x -- scalar, dimension of a training example = 42
    n_y -- scalar, dimension of output = 1
    
    """
    
    X = tf.placeholder(tf.float32, shape=[n_x,None], name='X')
    Y = tf.placeholder(tf.float32, shape=[n_y,None], name='Y')    
    return X, Y

In [9]:
def initialize_parameters():
    """
    Initializes parameters to build a neural network with tensorflow. The shapes are:
                        W1 : [30, 25]
                        b1 : [30, 1]
                        W2 : [25, 60]
                        b2 : [25, 1]
                        W3 : [60, 60]
                        b3 : [60, 1]
                        W4 : [60,60]
                        b4 : [60,1]
                        W5 : [60,60]
                        b5 : [60,1]
                        W6 : [60,60]
                        b6 : [60,1]
                        W7 : [1,60]
                        b7 : [1,1]
    """
    
    tf.set_random_seed(1)   
        
    W1 = tf.get_variable('W1', [25, 42], initializer=tf.contrib.layers.xavier_initializer(seed = 1))
    b1 = tf.get_variable('b1', [25, 1], initializer=tf.zeros_initializer())
    W2 = tf.get_variable('W2', [60, 25], initializer=tf.contrib.layers.xavier_initializer(seed = 1))
    b2 = tf.get_variable('b2', [60, 1], initializer=tf.zeros_initializer())
    W3 = tf.get_variable('W3', [60, 60], initializer=tf.contrib.layers.xavier_initializer(seed = 1))
    b3 = tf.get_variable('b3', [60, 1], initializer=tf.zeros_initializer())
    W4 = tf.get_variable('W4', [60, 60], initializer=tf.contrib.layers.xavier_initializer(seed = 1))
    b4 = tf.get_variable('b4', [60, 1], initializer=tf.zeros_initializer())
    W5 = tf.get_variable('W5', [60, 60], initializer=tf.contrib.layers.xavier_initializer(seed = 1))
    b5 = tf.get_variable('b5', [60 , 1], initializer=tf.zeros_initializer())
    W6 = tf.get_variable('W6', [60, 60], initializer=tf.contrib.layers.xavier_initializer(seed = 1))
    b6 = tf.get_variable('b6', [60 , 1], initializer=tf.zeros_initializer())
    W7 = tf.get_variable('W7', [1, 60], initializer=tf.contrib.layers.xavier_initializer(seed = 1))
    b7 = tf.get_variable('b7', [1, 1], initializer=tf.zeros_initializer())

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3,
                  "W4": W4,
                  "b4": b4,
                  "W5": W5,
                  "b5": b5,
                  "W6": W6,
                  "b6": b6,
                  "W7": W7,
                  "b7": b7}
    
    return parameters

In [10]:
def forward_propagation(X, parameters):
    """
    Implements the forward propagation for the model: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SOFTMAX
    
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3"
                  the shapes are given in initialize_parameters

    Returns:
    Z3 -- the output of the last LINEAR unit
    """
    
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    W4 = parameters['W4']
    b4 = parameters['b4']
    W5 = parameters['W5']
    b5 = parameters['b5']
    W6 = parameters['W6']
    b6 = parameters['b6']
    W7 = parameters['W7']
    b7 = parameters['b7']
    X = tf.transpose(X)
    print(W1)
    print(X.shape)
    
    Z1 = tf.add(tf.matmul(W1,X),b1)
    A1 = tf.nn.elu(Z1)
    print(A1.shape)
    Z2 = tf.add(tf.matmul(W2,A1),b2)
    A2 = tf.nn.elu(Z2)
    print(A2.shape)
    Z3 = tf.add(tf.matmul(W3,A2),b3)
    A3 = tf.nn.elu(Z3)
    
    Z4 = tf.add(tf.matmul(W4 , A3) , b4)
    A4 =  tf.nn.elu(Z4)
    
    Z5 = tf.add(tf.matmul(W5 , A4) , b5)
    A5 = tf.nn.elu(Z5)
    
    Z6 =  tf.add(tf.matmul(W6 , A5) , b6)
    A6 = tf.nn.elu(Z6)
    
    Z7 = tf.add(tf.matmul(W7 , A6 ) , b7)
    print("Z7 shape : ",Z7.shape)
    return Z7

In [11]:
def compute_cost(Z7, Y):
    """
    Computes the cost
    
    Arguments:
    Z3 -- output of forward propagation (output of the last LINEAR unit), of shape (1, number of examples)
    Y -- "true" labels vector placeholder, same shape as Z3
    
    Returns:
    cost - Tensor of the cost function
    """
    
    # to fit the tensorflow requirement for tf.nn.sigmoid_cross_entropy_with_logits(...,...)
    logits = tf.transpose(Z7)
    #labels = tf.transpose(Y)
    labels = Y
    ### START CODE HERE ### (1 line of code)
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels))
    ### END CODE HERE ###
    
    return cost

In [12]:
def calculate_accuracy(z, y):
    z_array = np.array(z).T
    y_array = np.array(y).T
    yhat_labels = (z_array > 0.5).astype(np.int)
    correct_labels = np.sum(y_array == yhat_labels)
    accuracy = correct_labels/y_array.shape[0]
    return accuracy

In [ ]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.0001, num_epochs = 3000, print_cost = True):
    """
    Implements a three-layer tensorflow neural network: LINEAR->RELU->LINEAR->RELU->LINEAR->SOFTMAX.
    
    Arguments:
    X_train -- training set, of shape (input size = 10, number of training examples = 72161)
    Y_train -- test set, of shape (output size = 2, number of training examples = 72161)
    X_test -- training set, of shape (input size = 10, number of training examples = 48108)
    Y_test -- test set, of shape (output size = 2, number of test examples = 48108)
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    print_cost -- True to print the cost every 100 epochs
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep consistent results
    seed = 3                                          # to keep consistent results
    (n_x, m) = X_train.shape                          # (n_x: input size, m : number of examples in the train set)
    n_y = Y_train.shape[0]                            # n_y : output size
    costs = []                                        # To keep track of the cost
    train_acc = []                                    # To keep track of the training accuracy
    test_acc = []                                     # To keep track of the test accuracy
    #saver = tf.train.Saver(parameters)  

    ### START CODE HERE ### (1 line)
    #X, Y = create_placeholders(n_x, n_y)
    X = tf.placeholder(tf.float32, shape= [25,42] , name = "X")
    Y = tf.placeholder(tf.float32 , shape= [25,1] , name="Y")
    ### END CODE HERE ###
    # Initialize parameters
    ### START CODE HERE ### (1 line)
    parameters = initialize_parameters()
    ### END CODE HERE ###
    
    # Forward propagation: Build the forward propagation in the tensorflow graph
    ### START CODE HERE ### (1 line)
    Z7 = forward_propagation(X, parameters)
    ### END CODE HERE ###
    
    # Cost function: Add cost function to tensorflow graph
    ### START CODE HERE ### (1 line)
    cost = compute_cost(Z7, Y)
    ### END CODE HERE ###
    
    # Backpropagation: Define the tensorflow optimizer. Use GradientDescentOptimizer.
    ### START CODE HERE ### (1 line)
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    ### END CODE HERE ###
    
    # Initialize all the variables
    init = tf.global_variables_initializer()

    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        #saver = tf.train.Saver(parameters)
        # Run the initialization
        sess.run(init)

        split_X_train = np.split(X_train, len(X_train) / batch_size)
        split_Y_train = np.split(Y_train , len(Y_train) / batch_size)
        
        split_X_test = np.split(X_test, len(X_test) / batch_size)
        split_Y_test = np.split(Y_test, len(Y_test) / batch_size)
        # Do the training loop
        for epoch in range(num_epochs):
            seed = seed + 1
            
            #offset = (epoch * batch_size) % (82300 - batch_size)
            
            #batch_data = X_train[offset:(offset + batch_size), :]
            #batch_labels = Y_train[offset:(offset + batch_size), :]
            #print(batch_data.shape)
            
            for batch in 
     
            #_, c = sess.run([optimizer, cost], feed_dict={X: X_train, Y: Y_train})
            _, c = sess.run([optimizer, cost], feed_dict={X: batch_data, Y: batch_labels})
            ### END CODE HERE ###
            
            epoch_cost = c                # Defines a cost related to an epoch
            
            test_batch_data = X_test[offset:(offset + batch_size), :]
            test_batch_labels = Y_test[offset:(offset + batch_size), :]
            
            epoch_train_Z = sess.run(Z7, feed_dict={X: batch_data})
            epoch_train_accuracy = calculate_accuracy(epoch_train_Z, batch_label)
            epoch_test_Z = sess.run(Z7, feed_dict={X: test_batch_data})
            epoch_test_accuracy = calculate_accuracy(epoch_test_Z, test_batch_labels)
            
            # Print the cost every epoch
            if print_cost == True and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)
                train_acc.append(epoch_train_accuracy)
                test_acc.append(epoch_test_accuracy)
            
            #if(epoch % 1000 == 0):
             #   saver.save(session, checkpoint_string + 'sess.ckpt', global_step=epoch)

                
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()
        
        # plot train vs. test accuracy over training epochs
        plt.figure(figsize=(13,7))
        plt.plot(np.squeeze(train_acc), color="red")
        plt.plot(np.squeeze(test_acc), color="blue")
        plt.ylabel('Accuracy')
        plt.xlabel('iterations (per tens)')
        plt.title("Training Accuracy (Red) vs. Test Accuracy (Blue) Trend")
        plt.show()
        
        # lets save the parameters in a variable
        parameters = sess.run(parameters)
        print ("Parameters have been trained!")

        # Accuracy
        train_Z = sess.run(Z7, feed_dict={X: X_train})
        train_accuracy = calculate_accuracy(train_Z, Y_train)
        test_Z = sess.run(Z7, feed_dict={X: X_test})
        test_accuracy = calculate_accuracy(test_Z, Y_test)

        print ("Train Accuracy:", train_accuracy)
        print ("Test Accuracy:", test_accuracy)
        

        return parameters

In [ ]:
import time
start_time = time.time()
print(time.asctime( time.localtime(time.time()) ))
parameters = model(X_train, Y_train, X_test, Y_test, num_epochs = 3000)
end_time = time.time() - start_time
print("--- %s seconds ---" % (time.time() - start_time))
print (time.asctime( time.localtime(time.time()) ))